# Modelo linguístico

Modelos Seq2Seq mapeiam sequências de tokens para outras sequências de tokens. Em geral, eles têm um par encoder-decoder. O encoder mapeia uma sequência inteira para 

In [79]:
from keras.layers import Input, Dense, Activation, TimeDistributed, Softmax, TextVectorization, Reshape, RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding, Concatenate, GlobalAveragePooling1D, LSTM, Multiply
from keras.models import Model
import tensorflow as tf
import keras
import numpy as np

In [3]:
# Refazer estrutura do dataset: uma frase por arquivo, arquivos em um diretorio
import pandas as pd
import os
import re
from tqdm import tqdm

#df = pd.read_csv('datasets/IMDB Dataset.csv')
#DATASET_DIR = './frases_imdb/'
DATASET_DIR = './datasets/frases/'
#DATASET_DIR = './datasets/frases_classificacao/'


In [4]:

# n_texto = 0
# for i in tqdm(range(len(df))):
#     texto = df.iloc[i]['review']
#     #print(texto)
#     texto = texto.replace('<br />', '.')
#     #texto = re.sub(r'[\n \t][\n \t][\n \t]+', '.', texto)
#     texto = re.split(r'[.!?\n\t-]', texto)
#     #print(texto)
#     for t in texto:

#         t = t.lstrip().rstrip()
#         if len(t) > 5:
#             #print(t)
#             fname = 'review_' + str(n_texto) + '.txt'
#             with open(DATASET_DIR + fname, 'w') as f:
#                 f.write(t)
#             n_texto += 1

# print("Criei textos:", n_texto)


In [10]:
# Ler um dataset e fazer batches
from tensorflow.keras.utils import text_dataset_from_directory

dataset = text_dataset_from_directory(
    DATASET_DIR,
    labels=None,
    label_mode='int',
    class_names=None,
    batch_size=2048,
    max_length=None,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    follow_links=False
)

Found 22516 files belonging to 1 classes.


In [84]:
from keras.layers import Input, TextVectorization
from keras.models import Model
vocab_size = 5000
vectorize_layer = TextVectorization(max_tokens=vocab_size, output_sequence_length=10)
vectorize_layer.adapt(dataset)

In [85]:
print(vectorize_layer.get_vocabulary())

['', '[UNK]', 'the', 'and', 'of', 'a', 'to', 'in', 'is', 'that', 'br', 'it', 'with', 'as', 'this', 'i', 'for', 'but', 'was', 'on', 'his', 'film', 'by', 'movie', 'who', 'are', 'he', 'not', 'be', 'have', 'from', 'you', 'at', 'one', 'an', 'her', 'they', 'all', 'its', 'or', 'like', 'so', 'which', 'some', 'about', 'out', 'has', 'when', 'their', 'just', 'more', 'up', 'if', 'what', 'there', 'would', 'even', 'into', 'time', 'she', 'very', 'only', 'had', 'story', 'because', 'were', 'good', 'can', 'other', 'no', 'him', 'than', 'we', 'people', 'them', 'being', 'most', 'been', 'will', 'see', 'get', 'where', 'my', 'after', 'much', 'really', 'first', 'how', 'films', 'characters', 'well', 'then', 'also', 'character', 'me', 'any', 'way', 'while', 'do', 'make', 'made', 'two', 'great', 'man', 'life', 'movies', 'many', 'could', 'off', 'little', 'scene', 'bad', 'over', 'such', 'through', 'think', 'plot', 'love', 'too', 'never', 'show', 'dont', 'scenes', 'these', 'still', 'know', 'seen', 'best', 'back', 't

In [86]:
# vocab = vectorize_layer.get_vocabulary()
# vocab = vocab[0:2] + ['[SOS]'] + vocab[2:-1]
# vectorize_layer.set_vocabulary(vocab)
# print(vectorize_layer.get_vocabulary())

In [87]:
def separar_ultimo_token(x):
    x_ = vectorize_layer(x)
    x_ = x_[:,:-1]
    y_ = x_[:,-1:]
    return x_, y_

In [100]:
def predict_word(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len-1,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding', mask_zero=True)(x)
    x = LSTM(latent_dim, kernel_initializer='glorot_uniform')(x)
    latent_rep = x
    x = Dense(vocab_size)(x)
    x = Softmax()(x)
    return Model(input_layer, x), Model(input_layer, latent_rep)

predictor, latent = predict_word(10, 15, vocab_size)
predictor.summary()
#opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
opt = keras.optimizers.Nadam(learning_rate=0.1)
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    ignore_class=1,
    name="sparse_categorical_crossentropy",
)

predictor.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])


Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 15)             75000     
                                                                 
 lstm_11 (LSTM)              (None, 15)                1860      
                                                                 
 dense_11 (Dense)            (None, 5000)              80000     
                                                                 
 softmax_11 (Softmax)        (None, 5000)              0         
                                                                 
Total params: 156,860
Trainable params: 156,860
Non-trainable params: 0
_________________________________________________________________


In [101]:
history = predictor.fit(dataset.map(separar_ultimo_token), epochs=20, verbose=1)


Epoch 1/20
11/11 [==============================] - 22s 270ms/step - loss: 6.7871 - accuracy: 0.0357
Epoch 2/20
11/11 [==============================] - 9s 304ms/step - loss: 4.6652 - accuracy: 0.1921
Epoch 3/20
11/11 [==============================] - 10s 315ms/step - loss: 3.2644 - accuracy: 0.4351
Epoch 4/20
11/11 [==============================] - 9s 287ms/step - loss: 2.5205 - accuracy: 0.5432
Epoch 5/20
11/11 [==============================] - 9s 296ms/step - loss: 2.0686 - accuracy: 0.6055
Epoch 6/20
11/11 [==============================] - 9s 285ms/step - loss: 1.7626 - accuracy: 0.6411
Epoch 7/20
11/11 [==============================] - 9s 290ms/step - loss: 1.5407 - accuracy: 0.6636
Epoch 8/20
11/11 [==============================] - 9s 295ms/step - loss: 1.3714 - accuracy: 0.6817
Epoch 9/20
11/11 [==============================] - 9s 290ms/step - loss: 1.2365 - accuracy: 0.6955
Epoch 10/20
11/11 [==============================] - 9s 290ms/step - loss: 1.1230 - accuracy: 0.70

In [102]:
pred = predictor.predict(vectorize_layer(["this movie is"])[:,:-1])
print(pred)
idx = tf.argmax(pred, axis=1)[0]
print(idx)
word = vectorize_layer.get_vocabulary()[idx]
print(word)

1/1 [==============================] - 4s 4s/step
[[4.9774087e-12 4.3837924e-12 1.5019800e-03 ... 2.0039743e-09
  3.8750864e-12 4.5052989e-12]]
tf.Tensor(8, shape=(), dtype=int64)
is


In [103]:
def predizer(entrada, numero_de_predicoes, modelo, vectorize_layer):
    frase = entrada
    for n in range(numero_de_predicoes):
        pred = modelo.predict(vectorize_layer([frase])[:,:-1])
        idx = tf.argmax(pred, axis=1)[0]
        word = vectorize_layer.get_vocabulary()[idx]
        frase = frase + " " + word
        print(word)
    return frase

predizer("this movie is", 2, predictor, vectorize_layer)

1/1 [==============================] - 0s 53ms/step
is
1/1 [==============================] - 0s 73ms/step
is


'this movie is is is'

In [106]:
def predizer2(entrada, numero_de_predicoes, modelo, vectorize_layer):
    frase = entrada
    contexto = frase # Contexto deslizante
    for n in range(numero_de_predicoes):
        pred = modelo.predict(vectorize_layer([contexto])[:,:-1])

        # Nao repetir palavras
        tentando = True
        while tentando:

            # Selectionar de k-best
            candidatos = tf.math.top_k(pred, k=10).indices[0,:]
            idx = np.random.choice(candidatos.numpy())
#            idx = tf.argmax(pred, axis=1)[0]
            word = vectorize_layer.get_vocabulary()[idx]
            if word in frase.split():
                pred[0][idx] = 0
            else:
                tentando = False
                
        frase = frase + " " + word
        contexto = contexto + " " + word
        #print(frase)
        contexto = ' '.join(frase.split()[1:])
        print(word)
    return frase

predizer2("i like how", 10, predictor, vectorize_layer)

1/1 [==============================] - 0s 81ms/step
but
1/1 [==============================] - 0s 41ms/step
or
1/1 [==============================] - 0s 55ms/step
off
1/1 [==============================] - 0s 52ms/step
chance
1/1 [==============================] - 0s 100ms/step
visual
1/1 [==============================] - 0s 41ms/step
100
1/1 [==============================] - 0s 44ms/step
hero
1/1 [==============================] - 0s 64ms/step
cannot
1/1 [==============================] - 0s 46ms/step
footage
1/1 [==============================] - 0s 83ms/step
came


'i like how but or off chance visual 100 hero cannot footage came'